In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, LSTM, TimeDistributed
from tensorflow.keras.optimizers import Adam

In [ ]:
data = pd.read_csv('Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv')

In [ ]:
selected_features = [
    ' Flow Duration', 'Total Length of Fwd Packets', 
    ' Fwd Packet Length Mean', 'Flow Bytes/s', ' Flow Packets/s', 
    ' Fwd IAT Mean', ' Bwd IAT Mean', ' Average Packet Size', ' SYN Flag Count', ' Total Backward Packets',
    ' ACK Flag Count', ' Packet Length Variance', ' Destination Port', ' Protocol'
]

In [ ]:
X = data[selected_features]
y = data[' Label']

In [ ]:
X.replace([np.inf, -np.inf], np.nan, inplace=True)

X.dropna(inplace=True)
y = y[X.index]  

In [ ]:
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_scaled = np.expand_dims(X_scaled, axis=2)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [ ]:
X_train.shape[1]

In [ ]:
model = Sequential()

In [ ]:
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(TimeDistributed(Flatten()))

model.add(LSTM(50, activation='relu'))

model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy', 'Precision', 'Recall'])


In [ ]:
model.fit(X_train, y_train, epochs=25, batch_size=32, validation_split=0.2)

In [ ]:
model.save('modelCNN.h5')

In [ ]:
loss, accuracy, precision, recall = model.evaluate(X_test, y_test)
print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')

In [ ]:
import time

In [ ]:
single_sample = np.array([X_test[0]])  

start_time = time.time()
prediction = model.predict(single_sample)
end_time = time.time()

time_taken = end_time - start_time
print(f'Time taken for a single prediction: {time_taken} seconds')

In [ ]:
num_samples = 100
samples = X_test[:num_samples] 

start_time = time.time()
start_p_time = time.process_time()
predictions = model.predict(samples)
end_p_time = time.process_time()
end_time = time.time()

time_taken = end_time - start_time
cpu_time_used = end_p_time - start_p_time
print(f'Time taken for predictions on {num_samples} samples: {time_taken} seconds // CPUTime: {cpu_time_used}')

In [ ]:
from memory_profiler import memory_usage

def model_training():
    num_samples = 1000
    samples = X_test[:num_samples] 
    start_time = time.time()
    predictions = model.predict(samples)
    pass
mem_usage = memory_usage(model_training)
print(f"Maximum memory usage: {max(mem_usage)} MiB")

In [ ]:
num_iterations = 100
total_time = 0

single_sample = np.array([X_test[0]])

for _ in range(num_iterations):
    start_time = time.time()
    predictions = model.predict(single_sample)
    end_time = time.time()
    total_time += end_time - start_time

average_inference_time = total_time / num_iterations
print(f"Average inference time: {average_inference_time} seconds")